In [1]:
import numpy as np
import os
from  matplotlib import pyplot as plt
import shutil
import re
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
import string
import json 
from time import time
import pickle 
import torchvision
import torch.nn as nn
import torch
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from torchvision import transforms
from PIL import Image
from utils import save_checkpoint, load_checkpoint, print_examples
from imageCaptionModel import CNNtoRNN
from torch.nn.utils.rnn import pad_sequence
from torch import optim 
from torchvision.models import EfficientNet_B1_Weights
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
from sklearn.model_selection import train_test_split
import collections
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset
import wandb
import os
from cv2 import threshold
import tqdm 

c:\Users\hosma\anaconda3\envs\torchEnv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [3]:
if torch.cuda.is_available(): 
    device = "cuda:0" 
else: 
    device = "cpu" 

torch.backends.cudnn.benchmark = True
load_model = False
save_model = True

##Dataset

In [4]:
efficentB1Transform = EfficientNet_B1_Weights.IMAGENET1K_V2.transforms()

In [5]:
from get_loader import get_loader


train_loader, dataset = get_loader(
        root_folder="Flickr8K/Images",
        annotation_file="Flickr8K/captions.txt",
        transform=efficentB1Transform,
        num_workers=2,
    )

In [6]:
image, captions = next(iter(train_loader))

In [7]:
captions

tensor([[   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   4,  149,    4,  178,    4,    4,    4,   10,    4,    6,    4,    4,
           50,   71,    4,   50,    4,    4,    4,    4,    4,    4,    4,    4,
            4,    4,    4,    4,    4,   10,   14,    4],
        [   9, 1874,   14,  453,    9,   20,   14,   80,   56,   69,   14,   82,
           97,   61,   61,   68,    6,   14,   30,    6,  525,   30,   30,    9,
          191,  638,  183,   80,  464,   50,   17,    6],
        [  43,   13,    8,  108,    7,    6,   12,   17,    8,    8,   17,  291,
           34,   97,   43,  153,  624,   17,   16,  108, 1112,    6,    6,    7,
           12, 1122,   79, 1911,  384,   97,   46,   75],
        [ 173,  711,  272, 1627,  582,  442,    4,   79,    4,   31,    3,   13,
           69,    8,   13,   35,    4, 

In [8]:
embed_size=256
hidden_size=256
num_layers = 1
vocab_size = len(dataset.vocab)
learning_rate = 3e-4
num_epochs = 100
model = CNNtoRNN(embed_size,hidden_size,vocab_size,num_layers).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

c:\Users\hosma\anaconda3\envs\torchEnv\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
c:\Users\hosma\anaconda3\envs\torchEnv\lib\site-packages\torch\nn\modules\module.py:1150: UserWarning: expandable_segments not supported on this platform (Triggered internally at ..\c10/cuda/CUDAAllocatorConfig.h:30.)
  return t.to(device, dtype if t.is_floating_point() or t.is_complex() else None, non_blocking)


In [9]:
wandb.login()

def train(model, train_dataloader, epochs=100, learning_rate=5e-5, save_path='./Models/'):
    # WandB Başlat
    wandb.init(project="Image Caption",settings=wandb.Settings(start_method="thread"))
    wandb.watch(model, log="all")

    step = 0

    if load_model:
        step = load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)
    
    model.train()

    # Modeli Eğitim Moduna Al
    model.train()
    for epoch in range(epochs):

        if save_model:
            checkpoint = {
                "state_dict":model.state_dict(),
                "optimizer":optimizer.state_dict(),
                "step":step
            }

            save_checkpoint(checkpoint)
        
        print(f"Epoch {epoch + 1}/{epochs}")

        epoch_loss = 0
        print("Training")
        model.train()
        for images, captions in tqdm.tqdm(train_dataloader):
            
            

            optimizer.zero_grad()

            # Veriyi ve hedefi al
            images = images.to(device)
            captions = torch.tensor(captions).to(device)

            outputs = model(images, captions[:-1])
            loss = criterion(
                outputs.reshape(-1, outputs.shape[2]), captions.reshape(-1)
            )

            
            step += 1

            optimizer.zero_grad()
            loss.backward(loss)
            optimizer.step()

            epoch_loss += loss.item()

        # Ortalama Eğitim Kaybını Loglama
        avg_epoch_loss = epoch_loss / len(train_dataloader)
        wandb.log({"epoch_loss": avg_epoch_loss})

      

        # Modeli Kaydetme
        checkpoint_path = os.path.join(save_path, f"epoch_{epoch + 1}.pt")
        torch.save(model.state_dict(), checkpoint_path)
        wandb.save(checkpoint_path)

    print("Eğitim Tamamlandı")
    wandb.finish()



# Eğitim Fonksiyonunu Çağırma
#train(model, train_loader)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: h-osmankarabulut. Use `wandb login --relogin` to force relogin


In [10]:
model.load_state_dict(torch.load("Models\epoch_100.pt"))

<All keys matched successfully>

In [11]:
print_examples(model, device, efficentB1Transform, dataset)

Example 1 CORRECT: Dog on a beach by the ocean
Example 1 OUTPUT: <SOS> a dog is running through the water . <EOS>
Example 2 CORRECT: Child holding red frisbee outdoors
Example 2 OUTPUT: <SOS> a little girl in a pink dress is playing with a red ball . <EOS>
Example 3 CORRECT: Bus driving by parked cars
Example 3 OUTPUT: <SOS> a man in a red shirt is standing in front of a skyscraper . <EOS>
Example 4 CORRECT: A small boat in the ocean
Example 4 OUTPUT: <SOS> a dog is running through the snow . <EOS>
Example 5 CORRECT: A cowboy riding a horse in the desert
Example 5 OUTPUT: <SOS> a man in a black shirt is standing in front of a white building . <EOS>
